# Analyze Vector DB

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

## Specify the embedding model and vector DB

In [ ]:
# ========== CONFIGURATION ==========
# Change this to analyze different databases
DATABASE_OPTIONS = {
    "qa": "../../data/vectordbs/qa_tool/",
    "code_gen": "../../data/vectordbs/code_gen/",
    "custom": "my_vector_store"  # For ad-hoc testing
}

ANALYZE_DB = "qa"  # <<< Change this to switch databases
# ===================================

database_loc = DATABASE_OPTIONS[ANALYZE_DB]
print(f"Analyzing database: {ANALYZE_DB}")
print(f"Path: {database_loc}")

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

vectorstore = Chroma(persist_directory=database_loc,
      embedding_function=embedding_model)

### (optional) Print the contents

In [ ]:
all_docs = vectorstore.get()['documents']

print(f"docs: {len(all_docs)}")

# for idx, doc in enumerate(all_docs):
#     print(f"Document {idx + 1}:")
#     print(doc)
#     print("-" * 80)

## Run a similarity search

In [ ]:
from typing import List
from langchain_core.runnables import chain
from langchain_core.documents import Document

@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vectorstore.similarity_search_with_score(query, k=8))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [ ]:
phrase = "What are slices?"
embedding = HuggingFaceEmbeddings().embed_query(phrase)

results = retriever.invoke(phrase)
#print(results)

for result in results:
    print(result.metadata)